# 1. Install Dependencies

In [ ]:
# Install required libraries
!pip install datasets transformers evaluate optuna peft
!apt-get install git-lfs

In [ ]:
# Load data
from datasets import load_dataset
imdb = load_dataset("imdb")
print(imdb)

train_dataset = imdb['train'].shuffle(seed=42)
test_dataset = imdb['test'].shuffle(seed=42)

# For hyperparameter tuning
val_split_ratio = 0.3
val_size = int(len(imdb["train"]) * val_split_ratio)

val_dataset_ht = train_dataset.select([i for i in list(range(val_size))])
train_dataset_ht = train_dataset.select([i for i in list(range(val_size, len(imdb["train"])))])

print(len(test_dataset))
print(len(val_dataset_ht))
print(len(train_dataset_ht))

In [ ]:
import torch
import optuna
from transformers import Trainer, TrainingArguments
from transformers.models.auto import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import evaluate

torch.manual_seed(42)

CHECKPOINT = "microsoft/deberta-v3-base"

tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

train_dataset_ht = train_dataset_ht.map(tokenize_function, batched=True)
val_dataset_ht = val_dataset_ht.map(tokenize_function, batched=True)





In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
config = AutoConfig.from_pretrained(CHECKPOINT, num_labels=2)

def objective(trial: optuna.Trial):
    model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, config=config)

    batch_size = trial.suggest_categorical('batch_size', [16, 32])
    learning_rate = trial.suggest_categorical('learning_rate', [5e-5, 3e-5, 2e-5])
    num_epochs = trial.suggest_categorical('num_epochs', [2, 3, 4])

    training_args = TrainingArguments(
        output_dir="./results",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs
        )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset_ht,
        eval_dataset=val_dataset_ht)

    trainer.train()
    # Evaluate the model on the validation dataset
    eval_result = trainer.evaluate()

    # Return validation loss (you can add other metrics here as needed)
    return eval_result["eval_loss"]


study = optuna.create_study(study_name='hp-search-deberta', direction='minimize')
study.optimize(func=objective, n_trials=5)

best_lr = float(study.best_params['learning_rate'])
best_batch_size = study.best_params['batch_size']
best_epoch = int(study.best_params['num_epochs'])

print(f"Best Learning Rate: {best_lr}")
print(f"Best Batch Size: {best_batch_size}")
print(f"Best Epochs: {best_epoch}")



In [ ]:
# Define evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
recall_metric = evaluate.load("recall")
precision_metric = evaluate.load("precision")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = f1_metric.compute(predictions=predictions, references=labels)["f1"]
    recall = recall_metric.compute(predictions=predictions, references=labels)["recall"]
    precision = precision_metric.compute(predictions=predictions, references=labels)["precision"]
    return {"accuracy": accuracy, "f1": f1, "recall": recall, "precision": precision}


model = AutoModelForSequenceClassification.from_pretrained(CHECKPOINT, num_labels=2)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=best_lr,
    per_device_train_batch_size=best_batch_size,
    per_device_eval_batch_size=best_batch_size,
    num_train_epochs=best_epoch
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
train_results = trainer.train()
print(train_results)

# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)


trainer.save_model("./best-deberta-model")